# jupyterlite-pyodide-lock demo

This demo shows some of the interactive features of `jupyterlite-pyodide-lock`.

> Learn more in the [project description](./README.md) or [history](./CHANGELOG.md).

## no `%pip` needed

As with `piplite`, the default package management approach for `jupyterlite-pyodide-kernel`, site owners can include custom wheels, or rely on runtime installation.

However, packages indexed by `piplite` _still_ require the `%pip` magic.

> _This is needed because there's no way to know how an `import`able name maps to a distribution name, much less a version, on PyPI, the Python Package Index._

With `jupyterlite-pyodide-lock`, packages that have been _locked_ ahead of time are available as if they had already been installed.

In [ ]:
import sys

print("before loading ipywidgets,", len(sys.modules), "modules are imported")

In [ ]:
import ipywidgets as W
import traitlets as T

print("ipywidgets version is", W.__version__)
s = [W.FloatSlider(description=x) for x in "abc"]
T.dlink((s[0], "value"), (s[1], "value"), lambda x: x / 2)
T.dlink((s[1], "value"), (s[2], "value"), lambda x: x + s[0].value)
display(*s)

In [ ]:
print("after loading ipywidgets,", len(sys.modules), "modules are imported")

## preloaded (not imported) packages

`pyodide_kernel`, the `ipykernel` shim, `ipython`, and all their further dependencies were _downloaded_, _cached_, and _installed_ while this `pyodide` virtual machine was initializing. 

> _This behavior can be extended to custom importable names with `PyodideLockAddon.extra_preload_packages`_

However, only the **absolute minimum** number of packages will be _imported_ before providing a site visitor an interactive computing experience.

## more reproducible

After being locked, packages will load the same way for every site visitor. This helps the site provides with the intended experience.

In [ ]:
import micropip
import json
from IPython.display import JSON, Markdown

In [ ]:
package = W.Select(description="package", rows=1)
frozen = W.Output()

def on_frozen_package(*_):
    frozen.clear_output()
    with frozen:
        if not package.value:
            display(Markdown("> pick a package!"))
        else:
            lock = json.loads(micropip.freeze())
            display(JSON(lock["packages"][package.value], expanded=True))

def update_lock(*_):
    lock = json.loads(micropip.freeze())
    package.options = tuple(sorted(lock["packages"]))
    on_frozen_package()


package.observe(on_frozen_package, "value")
update_lock()
package.value = "ipywidgets"

W.HBox([package, frozen])

## `%pip` is still available

Using the `%pip` magic will still work (in a portable way) meaning interactive exploration of the thousands of packages on PyPI are still possible in a form that can be downloaded or moved to a "standard" installation like MyBinder.

Any pure python package can still be installed and imported, as long as all of its dependencies are either _also_ pure python, included in the lock file, or have been built for `pyodide`.

In [ ]:
%pip install traittypes
import traittypes

traittypes.__version__